# Fullshot

In [37]:
from glob import glob 
import pandas as pd 
import numpy as np 
import os 

root   = '../../Dataset/MVtecAD'
Object = pd.Series(glob(os.path.join(root,'*/'))).apply(lambda x : x.split('/')[-2]).values
phase  = ['train','test','ground_truth']

img_dir_list = [] 

for obj in Object:
    for phs in phase:
        if phs == 'train':
            img_dirs = sorted(glob(os.path.join(root,obj,phs,'good','*.png')))
            img_dir_list.extend(img_dirs)
        if phs == 'test':
            for test_type in sorted(glob(os.path.join(root,obj,phase[1],'*/'))):
                img_dirs = sorted(glob(os.path.join(test_type,'*')))
                img_dir_list.extend(img_dirs)


df = pd.DataFrame(img_dir_list)
df.columns = ['image']
df['object'] = df['image'].apply(lambda x : x.split('/')[4])
df['split'] = df['image'].apply(lambda x : x.split('/')[5])
df['label'] = df['image'].apply(lambda x : x.split('/')[6])
df['mask'] = ''
gt_dirs = df[df['split'] == 'test']['image'].apply(lambda x : x.replace('test','ground_truth').replace('.png','_mask.png'))
gt_index = gt_dirs.index
df.loc[gt_index,'mask'] = gt_dirs
df['image'] = df['image'].apply(lambda x : x.split('MVtecAD')[-1][1:])
df['mask'] = df['mask'].apply(lambda x : x.split('MVtecAD')[-1][1:])
df.loc[(df['split']=='test') & (df['label']=='good'),'mask'] = ''
df['label1'] = df['label']
df['label'] = df['label'].apply(lambda x : 'normal' if x =='good' else 'anomaly')
df = df[['object','split','label','image','mask','label1']]
df.to_csv('./split_csv/1cls.csv',index=False)

# Fewshot 

In [45]:

def make_fewshot(df,shot_ratio=0.8):
    Objects = np.unique(df['object'])
    for obj in Objects:

        #before 
        train_normal_idx = df.loc[(df['split']=='train')&(df['object']==obj)].index
        test_normal_idx = df.loc[(df['split']=='test')&(df['object']==obj)&(df['label']=='normal')].index
        anomaly_idx = df.loc[(df['split']=='test')&(df['object']==obj)&(df['label']=='anomaly')].index
        normal_idx = np.append(train_normal_idx,test_normal_idx)
        #after
        ##normal 
        test_normal_idx = np.random.choice(normal_idx,int(len(normal_idx)*shot_ratio),replace=False)
        train_normal_idx = pd.Series(normal_idx)[pd.Series(normal_idx).apply(lambda x : x not in test_normal_idx)].values    
        ##anomaly 
        test_anomaly_idx = np.random.choice(anomaly_idx,int(len(anomaly_idx)*shot_ratio),replace=False)
        train_anomaly_idx = pd.Series(anomaly_idx)[pd.Series(anomaly_idx).apply(lambda x : x not in test_anomaly_idx)].values    
        
        df.loc[test_normal_idx,'split'] = 'test'
        df.loc[test_normal_idx,'label'] = 'normal'

        df.loc[train_normal_idx,'split'] = 'train'
        df.loc[train_normal_idx,'label'] = 'normal'

        df.loc[test_anomaly_idx,'split'] = 'test'
        df.loc[test_anomaly_idx,'label'] = 'anomaly'

        df.loc[train_anomaly_idx,'split'] = 'train'
        df.loc[train_anomaly_idx,'label'] = 'anomaly'
    return df 

df = pd.read_csv('./split_csv/1cls.csv')

adf = make_fewshot(df,0.8)
adf.to_csv('./split_csv/2cls_fewshot.csv',index=False)

# Highshot 

In [44]:

def make_fewshot(df,shot_ratio=0.8):
    Objects = np.unique(df['object'])
    for obj in Objects:

        #before 
        train_normal_idx = df.loc[(df['split']=='train')&(df['object']==obj)].index
        test_normal_idx = df.loc[(df['split']=='test')&(df['object']==obj)&(df['label']=='normal')].index
        anomaly_idx = df.loc[(df['split']=='test')&(df['object']==obj)&(df['label']=='anomaly')].index
        normal_idx = np.append(train_normal_idx,test_normal_idx)
        #after
        ##normal 
        test_normal_idx = np.random.choice(normal_idx,int(len(normal_idx)*shot_ratio),replace=False)
        train_normal_idx = pd.Series(normal_idx)[pd.Series(normal_idx).apply(lambda x : x not in test_normal_idx)].values    
        ##anomaly 
        test_anomaly_idx = np.random.choice(anomaly_idx,int(len(anomaly_idx)*shot_ratio),replace=False)
        train_anomaly_idx = pd.Series(anomaly_idx)[pd.Series(anomaly_idx).apply(lambda x : x not in test_anomaly_idx)].values    
        
        df.loc[test_normal_idx,'split'] = 'test'
        df.loc[test_normal_idx,'label'] = 'normal'

        df.loc[train_normal_idx,'split'] = 'train'
        df.loc[train_normal_idx,'label'] = 'normal'

        df.loc[test_anomaly_idx,'split'] = 'test'
        df.loc[test_anomaly_idx,'label'] = 'anomaly'

        df.loc[train_anomaly_idx,'split'] = 'train'
        df.loc[train_anomaly_idx,'label'] = 'anomaly'
    return df 

df = pd.read_csv('./split_csv/1cls.csv')

adf = make_fewshot(df,0.4)
adf.to_csv('./split_csv/2cls_highshot.csv',index=False)

In [1]:
import pandas as pd 

In [43]:
df = pd.read_csv('../ViSA/split_csv/1cls.csv')

In [45]:
df[(df['object']=='candle')&(df['split']=='train')]['label'].value_counts()

normal    900
Name: label, dtype: int64

In [11]:
df[df['split']=='train']['label'].value_counts()

normal    3629
Name: label, dtype: int64